In [1]:
import pandas as pd 

In [2]:
transfer= pd.DataFrame.from_csv('../GTFS_nyc_Subway/transfers.txt')
transfer  = transfer.reset_index()
transfer.head(2)

In [7]:
transfer['from_stop']=None
transfer['to_stop']=None

In [9]:
transfer.head(2)

,from_stop_id,to_stop_id,transfer_type,min_transfer_time,from_stop,to_stop
0,101,101,2,180,None,None
1,103,103,2,180,None,None


#### get the name of stations

In [10]:
stops = pd.read_csv('../GTFS_nyc_Subway/stops.txt').reset_index()

In [11]:
stops.head(1)

,index,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
0,0,101,NaN,Van Cortlandt Park - 242 St,NaN,40.889248,-73.898583,NaN,NaN,1,NaN


In [12]:
for i in range(len(transfer)):
    from_id = transfer.iloc[i,0]
    to_id = transfer.iloc[i,1]
    try:
        transfer.iloc[i,4] = stops[stops['stop_id']==from_id]['stop_name'].values[0]
    except IndexError:
        print from_id
        
    try:
        transfer.iloc[i,5] = stops[stops['stop_id']==to_id]['stop_name'].values[0]
    except IndexError:
        print to_id

In [13]:
transfer[transfer['from_stop']!=transfer['to_stop']].head(3)

,from_stop_id,to_stop_id,transfer_type,min_transfer_time,from_stop,to_stop
10,112,A09,2,180,168 St - Washington Hts,168 St
29,127,A27,2,300,Times Sq - 42 St,42 St - Port Authority Bus Terminal
37,132,L02,2,180,14 St,6 Av


#### so this transfer data includes those transfer between two stations

## get info about line-line

In [14]:
transfer['from_line']=None
transfer['to_line']=None

In [15]:
transfer.head(2)

,from_stop_id,to_stop_id,transfer_type,min_transfer_time,from_stop,to_stop,from_line,to_line
0,101,101,2,180,Van Cortlandt Park - 242 St,Van Cortlandt Park - 242 St,None,None
1,103,103,2,180,238 St,238 St,None,None


In [17]:
stations = pd.read_csv('../../data/nyc_subway_Stations_location_20170307.csv')

In [19]:
stations.head(2)

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude
0,1,1,R01,BMT,Astoria,Astoria - Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843


In [20]:
for i in range(len(transfer)):
    from_id = transfer.iloc[i,0]
    to_id = transfer.iloc[i,1]
    try:
        transfer.iloc[i,6] = stations[stations['GTFS Stop ID']==from_id]['Daytime Routes'].values[0]
    except IndexError:
        print i,'from_id: ',from_id
        
    try:
        transfer.iloc[i,7] = stations[stations['GTFS Stop ID']==to_id]['Daytime Routes'].values[0]
    except IndexError:
        print i,'to_id: ',to_id

207 to_id:  R09
241 to_id:  D13
264 to_id:  D20
332 from_id:  D13
333 from_id:  D13
333 to_id:  D13
344 from_id:  D20
345 from_id:  D20
345 to_id:  D20
553 from_id:  R09
554 from_id:  R09
554 to_id:  R09


In [71]:
transfer.loc[[206,207],['from_line']]='7'
transfer.loc[[553,554],['from_line']]='N W'

transfer.loc[[206,553],['to_line']]='7'
transfer.loc[[207,554],['to_line']]='N W'

transfer.loc[554,'min_transfer_time']=0

In [72]:
transfer.iloc[[206,207,553,554]]

,from_stop_id,to_stop_id,transfer_type,min_transfer_time,from_stop,to_stop,from_line,to_line
206,718,718,2,0,Queensboro Plaza,Queensboro Plaza,7,7
207,718,R09,2,0,Queensboro Plaza,Queensboro Plaza,7,N W
553,R09,718,2,0,Queensboro Plaza,Queensboro Plaza,N W,7
554,R09,R09,2,0,Queensboro Plaza,Queensboro Plaza,N W,N W


In [38]:
transfer.loc[[207,554],['to_line']]='7 N W'
transfer.loc[[553,554],['from_line']]='7 N W'

In [54]:
transfer.loc[[240,241],['from_line']]='A C'
transfer.loc[[332,333],['from_line']]='B D'

transfer.loc[[240,332],['to_line']]='A C'
transfer.loc[[241,333],['to_line']]='B D'

In [55]:
transfer.iloc[[240,241,332,333]]

,from_stop_id,to_stop_id,transfer_type,min_transfer_time,from_stop,to_stop,from_line,to_line
240,A12,A12,2,0,145 St,145 St,A C,A C
241,A12,D13,2,180,145 St,145 St,A C,B D
332,D13,A12,2,180,145 St,145 St,B D,A C
333,D13,D13,2,0,145 St,145 St,B D,B D


In [64]:
transfer.loc[[263,264],['from_line']]='A C E'
transfer.loc[[344,345],['from_line']]='B D F M'

transfer.loc[[263,344],['to_line']]='A C E'
transfer.loc[[264,345],['to_line']]='B D F M'

In [65]:
transfer.iloc[[263,264,344,345]]

,from_stop_id,to_stop_id,transfer_type,min_transfer_time,from_stop,to_stop,from_line,to_line
263,A32,A32,2,0,W 4 St,W 4 St,A C E,A C E
264,A32,D20,2,180,W 4 St,W 4 St,A C E,B D F M
344,D20,A32,2,180,W 4 St,W 4 St,B D F M,A C E
345,D20,D20,2,0,W 4 St,W 4 St,B D F M,B D F M


In [77]:
append=stations[stations['GTFS Stop ID'].isin(['A12','A32','718'])].copy()

In [88]:
stations[stations['GTFS Stop ID'].isin(['A12','A32','718'])]#['Daytime Routes']=['A C','A C E','7']

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude
149,151,151,A12,IND,8th Av - Fulton St,145 St,M,A B C D,Subway,40.824783,-73.944216
165,167,167,A32,IND,8th Av - Fulton St,W 4 St,M,A B C D E F M,Subway,40.732338,-74.000495
459,461,461,718,IRT,Flushing,Queensboro Plaza,Q,7 N W,Elevated,40.750582,-73.940202


In [91]:
stations.loc[[149,165,459],'Daytime Routes']=['A C','A C E','7']

In [81]:
append['Station ID']=None
append['Complex ID']=None
append['Division']=None
append['Line']=None

In [83]:
append['GTFS Stop ID']=['D13','D20','R09']

In [85]:
append['Daytime Routes']=['B D','B D F M','N W']

In [97]:
stations = pd.concat([stations,append])

In [98]:
stations.to_csv('../../data/nyc_subway_Stations_location_20170307.csv')

In [101]:
for i in range(len(transfer)):
    from_id = transfer.iloc[i,0]
    to_id = transfer.iloc[i,1]
    try:
        transfer.iloc[i,6] = stations[stations['GTFS Stop ID']==from_id]['Daytime Routes'].values[0]
    except IndexError:
        print i,'from_id: ',from_id
        
    try:
        transfer.iloc[i,7] = stations[stations['GTFS Stop ID']==to_id]['Daytime Routes'].values[0]
    except IndexError:
        print i,'to_id: ',to_id

In [131]:
transfer[transfer['from_stop_id']==transfer['to_stop_id']].head()

,from_stop_id,to_stop_id,transfer_type,min_transfer_time,from_stop,to_stop,from_line,to_line
0,101,101,2,180,Van Cortlandt Park - 242 St,Van Cortlandt Park - 242 St,1,1
1,103,103,2,180,238 St,238 St,1,1
2,104,104,2,180,231 St,231 St,1,1
3,106,106,2,180,Marble Hill - 225 St,Marble Hill - 225 St,1,1
4,107,107,2,180,215 St,215 St,1,1


In [112]:
transfer[transfer['from_stop']=='East Broadway']

,from_stop_id,to_stop_id,transfer_type,min_transfer_time,from_stop,to_stop,from_line,to_line
388,F16,F16,2,180,East Broadway,East Broadway,F,F


### It is so wierd!! Why the same track transfer still cost time?

In [225]:
def combine(f,t):
    for i in f:
        for j in t:
            yield (i,j)

In [246]:
trans=pd.DataFrame(columns=transfer.columns)
for r in transfer.index:
    f = transfer.iloc[r,-2].split(' ')
    t = transfer.iloc[r,-1].split(' ')
    for i,j in combine(f,t):
    
        df= transfer.iloc[r:r+1].copy()
        df.from_line=i
        df.to_line=j
        trans = pd.concat([trans,df])

In [252]:
trans = trans[trans['from_line']!=trans['to_line']]

In [257]:
trans.reset_index(drop=True).to_csv('../cleaned_data/transfer_without_swipe')

## dataframe stops and stations are different in following 3 stations

Conclusion:
1) Staten Island cancel "Arthur Kill", but use 'Nassau' and 'Atlantic' to replace
2) stops are more updated
3) N12 is the last station in coney island. B F N Q all ends there. 
   In stations, it is only known as "D43"
   In stops, it is known as "D43" and "N14", but they are of same geo-coding.

In [166]:
for i in stops.stop_id[::3].values:
    if i in stations['GTFS Stop ID'].values:
        pass
    else:
        print i

N12
S10
S12


In [188]:
stops[stops.stop_id.isin(['N12','S10','S12','D43'])]

,index,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
855,855,D43,NaN,Coney Island - Stillwell Av,NaN,40.577422,-73.981233,NaN,NaN,1,NaN
1287,1287,N12,NaN,S.B. Coney Island,NaN,40.577422,-73.981233,NaN,NaN,1,NaN
1431,1431,S10,NaN,Atlantic,NaN,40.515401,-74.245689,NaN,NaN,1,NaN
1437,1437,S12,NaN,Nassau,NaN,40.517812,-74.238373,NaN,NaN,1,NaN


In [176]:
stops[stops.stop_id.isin(['D43'])]

,index,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
855,855,D43,NaN,Coney Island - Stillwell Av,NaN,40.577422,-73.981233,NaN,NaN,1,NaN


In [174]:
stations[stations['GTFS Latitude']==40.577422]

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude
57,58,58,D43,BMT,Sea Beach / West End / Culver / Brighton,Coney Island - Stillwell Av,Bk,D F N Q,Viaduct,40.577422,-73.981233


In [183]:
stations[stations['Daytime Routes']=='SIR']

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude
472,501,501,S31,SIR,Staten Island,St George,SI,SIR,Open Cut,40.643748,-74.073643
473,502,502,S30,SIR,Staten Island,Tompkinsville,SI,SIR,At Grade,40.636949,-74.074835
474,503,503,S29,SIR,Staten Island,Stapleton,SI,SIR,Elevated,40.627915,-74.075162
475,504,504,S28,SIR,Staten Island,Clifton,SI,SIR,Elevated,40.621319,-74.071402
476,505,505,S27,SIR,Staten Island,Grasmere,SI,SIR,Open Cut,40.603117,-74.084087
477,506,506,S26,SIR,Staten Island,Old Town,SI,SIR,Embankment,40.596612,-74.087368
478,507,507,S25,SIR,Staten Island,Dongan Hills,SI,SIR,Embankment,40.588849,-74.096090
479,508,508,S24,SIR,Staten Island,Jefferson Av,SI,SIR,Embankment,40.583591,-74.103338
480,509,509,S23,SIR,Staten Island,Grant City,SI,SIR,Open Cut,40.578965,-74.109704
481,510,510,S22,SIR,Staten Island,New Dorp,SI,SIR,Open Cut,40.573480,-74.117210


## CLEAN UP NODES

In [276]:
stations= stations[['GTFS Stop ID','Borough','Daytime Routes']]

In [278]:
stations= stations.rename(columns={'GTFS Stop ID':'stop_id'})

In [280]:
stations.head(2)

,stop_id,Borough,Daytime Routes
0,R01,Q,N W
1,R03,Q,N W


In [270]:
stops= stops[['stop_id','stop_name','stop_lat','stop_lon']][::3]

In [283]:
stops=stops.merge(stations,on='stop_id',how='left')

In [310]:
stops[(stops.Borough>'A')==False]

,stop_id,stop_name,stop_lat,stop_lon,Borough,Daytime Routes


In [305]:
stops= stops[stops['stop_id']!='N12']

In [309]:

stops.loc[477,'Borough']='SI'
stops.loc[479,'Borough']='SI'

stops.loc[477,'Daytime Routes']='SIR'
stops.loc[479,'Daytime Routes']='SIR'

//anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [311]:
stops.to_csv('../cleaned_data/nyc_subway_stations_with_lines')

In [321]:
stops=stops.reset_index(drop=True)

In [319]:
stops.iloc[34:40]

,stop_id,stop_name,stop_lat,stop_lon,Borough,Daytime Routes
34,137,Chambers St,40.715478,-74.009266,M,1 2 3
35,138,Cortlandt St,40.711835,-74.012188,M,1
36,139,Rector St,40.707513,-74.013783,M,1
37,140,South Ferry Loop,40.701411,-74.013205,M,1
38,201,Wakefield - 241 St,40.903125,-73.850620,Bx,2
39,204,Nereid Av,40.898379,-73.854376,Bx,2 5


In [327]:
st=pd.DataFrame(columns=stops.columns)
for r in stops.index:
    routes = stops.iloc[r,-1].split(' ')
    for i in routes:
    
        df= stops.iloc[r:r+1].copy()
        df['Daytime Routes']=i
        st = pd.concat([st,df])

In [330]:
st.reset_index(drop=True).to_csv('../cleaned_data/nyc_subway_stations_with_lines_2(splited row)')

In [332]:
st.head(10)

,stop_id,stop_name,stop_lat,stop_lon,Borough,Daytime Routes
0,101,Van Cortlandt Park - 242 St,40.889248,-73.898583,Bx,1
1,103,238 St,40.884667,-73.900870,Bx,1
2,104,231 St,40.878856,-73.904834,Bx,1
3,106,Marble Hill - 225 St,40.874561,-73.909831,M,1
4,107,215 St,40.869444,-73.915279,M,1
5,108,207 St,40.864621,-73.918822,M,1
6,109,Dyckman St,40.860531,-73.925536,M,1
7,110,191 St,40.855225,-73.929412,M,1
8,111,181 St,40.849505,-73.933596,M,1
9,112,168 St - Washington Hts,40.840556,-73.940133,M,1


In [342]:
stops[stops['stop_name']=='Times Sq - 42 St']

,stop_id,stop_name,stop_lat,stop_lon,Borough,Daytime Routes
24,127,Times Sq - 42 St,40.755290,-73.987495,M,1 2 3
174,725,Times Sq - 42 St,40.755477,-73.987691,M,7
177,902,Times Sq - 42 St,40.755983,-73.986229,M,S
444,R16,Times Sq - 42 St,40.754672,-73.986754,M,N Q R W


#### stop_id is the unique id for each node. Each track at each station has one single node. For example, Time Sq - 42 St, has 4 tracks, and 4 nodes as above.